In [54]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 14: Chocolate Charts ---</h2><p>You finally have a chance to look at all of the produce moving around. Chocolate, cinnamon, mint, chili peppers, nutmeg, vanilla... the Elves must be growing these plants to <span title="Please do not use a programming puzzle as a recipe for hot chocolate. I cannot guarantee your safety.">make</span> <em>hot chocolate</em>! As you realize this, you hear a conversation in the distance. When you go to investigate, you discover two Elves in what appears to be a makeshift underground kitchen/laboratory.</p>
<p>The Elves are trying to come up with the ultimate hot chocolate recipe; they're even maintaining a scoreboard which tracks the quality <em>score</em> (<code>0</code>-<code>9</code>) of each recipe.</p>
<p>Only two recipes are on the board: the first recipe got a score of <code>3</code>, the second, <code>7</code>. Each of the two Elves has a <em>current recipe</em>: the first Elf starts with the first recipe, and the second Elf starts with the second recipe.</p>
<p>To create new recipes, the two Elves combine their current recipes.  This creates new recipes from the <em>digits of the sum</em> of the current recipes' scores.  With the current recipes' scores of <code>3</code> and <code>7</code>, their sum is <code>10</code>, and so two new recipes would be created: the first with score <code>1</code> and the second with score <code>0</code>. If the current recipes' scores were <code>2</code> and <code>3</code>, the sum, <code>5</code>, would only create one recipe (with a score of <code>5</code>) with its single digit.</p>
<p>The new recipes are added to the end of the scoreboard in the order they are created.  So, after the first round, the scoreboard is <code>3, 7, 1, 0</code>.</p>
<p>After all new recipes are added to the scoreboard, each Elf picks a new current recipe.  To do this, the Elf steps forward through the scoreboard a number of recipes equal to <em>1 plus the score of their current recipe</em>. So, after the first round, the first Elf moves forward <code>1 + 3 = 4</code> times, while the second Elf moves forward <code>1 + 7 = 8</code> times. If they run out of recipes, they loop back around to the beginning. After the first round, both Elves happen to loop around until they land on the same recipe that they had in the beginning; in general, they will move to different recipes.</p>
<p>Drawing the first Elf as parentheses and the second Elf as square brackets, they continue this process:</p>
<pre><code>(3)[7]
(3)[7] 1  0 
 3  7  1 [0](1) 0 
 3  7  1  0 [1] 0 (1)
(3) 7  1  0  1  0 [1] 2 
 3  7  1  0 (1) 0  1  2 [4]
 3  7  1 [0] 1  0 (1) 2  4  5 
 3  7  1  0 [1] 0  1  2 (4) 5  1 
 3 (7) 1  0  1  0 [1] 2  4  5  1  5 
 3  7  1  0  1  0  1  2 [4](5) 1  5  8 
 3 (7) 1  0  1  0  1  2  4  5  1  5  8 [9]
 3  7  1  0  1  0  1 [2] 4 (5) 1  5  8  9  1  6 
 3  7  1  0  1  0  1  2  4  5 [1] 5  8  9  1 (6) 7 
 3  7  1  0 (1) 0  1  2  4  5  1  5 [8] 9  1  6  7  7 
 3  7 [1] 0  1  0 (1) 2  4  5  1  5  8  9  1  6  7  7  9 
 3  7  1  0 [1] 0  1  2 (4) <em>5  1  5  8  9  1  6  7  7  9</em>  2 
</code></pre>
<p>The Elves think their skill will improve after making a few recipes (your puzzle input). However, that could take ages; you can speed this up considerably by identifying <em>the scores of the ten recipes</em> after that.  For example:</p>
<ul>
<li>If the Elves think their skill will improve after making <code>9</code> recipes, the scores of the ten recipes <em>after</em> the first nine on the scoreboard would be <code>5158916779</code> (highlighted in the last line of the diagram).</li>
<li>After <code>5</code> recipes, the scores of the next ten would be <code>0124515891</code>.</li>
<li>After <code>18</code> recipes, the scores of the next ten would be <code>9251071085</code>.</li>
<li>After <code>2018</code> recipes, the scores of the next ten would be <code>5941429882</code>.</li>
</ul>
<p><em>What are the scores of the ten recipes immediately after the number of recipes in your puzzle input?</em></p>
</article>


In [55]:
tests = [
    {
        "name": "Example 1",
        "n": 9,
        "expected": "5158916779",
    },
    {
        "name": "Example 2",
        "n": 5,
        "expected": "0124515891",
    },
    {
        "name": "Example 3",
        "n": 18,
        "expected": "9251071085",
    },
    {
        "name": "Example 4",
        "n": 2018,
        "expected": "5941429882",
    },
]


def score(n: int, do_print: bool = False) -> int:
    recipes = [3, 7]
    current_recipes = [0, 1]
    while len(recipes) < n + 11:
        if do_print:
            print(
                "".join(
                    (
                        f"({r})"
                        if i == current_recipes[0]
                        else f"[{r}]" if i == current_recipes[1] else f" {r} "
                    )
                    for i, r in enumerate(recipes)
                )
            )
        recipes.extend(
            v
            for i, v in enumerate(divmod(sum(recipes[i] for i in current_recipes), 10))
            if i != 0 or v != 0
        )
        current_recipes = [(i + 1 + recipes[i]) % len(recipes) for i in current_recipes]

    if do_print:
        print(
            "".join(
                (
                    f"({r})"
                    if i == current_recipes[0]
                    else f"[{r}]" if i == current_recipes[1] else f" {r} "
                )
                for i, r in enumerate(recipes)
            )
        )

    return "".join(str(i) for i in recipes[n : n + 10])


run_tests_params(score, tests)


Test Example 1 passed, for score.
Test Example 2 passed, for score.
Test Example 3 passed, for score.
Test Example 4 passed, for score.
Success


In [56]:
print(f"Part I: {score(846021)}")

Part I: 5482326119


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>5482326119</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As it turns out, you got the Elves' plan backwards.  They actually want to know how many recipes appear on the scoreboard to the left of the first recipes whose scores are the digits from your puzzle input.</p>
<ul>
<li><code>51589</code> first appears after <code>9</code> recipes.</li>
<li><code>01245</code> first appears after <code>5</code> recipes.</li>
<li><code>92510</code> first appears after <code>18</code> recipes.</li>
<li><code>59414</code> first appears after <code>2018</code> recipes.</li>
</ul>
<p><em>How many recipes appear on the scoreboard to the left of the score sequence in your puzzle input?</em></p>
</article>

</main>


In [57]:
tests_II = [
    {
        "name": "Example 1",
        "s": "51589",
        "expected": 9,
    },
    {
        "name": "Example 2",
        "s": "01245",
        "expected": 5,
    },
    {
        "name": "Example 3",
        "s": "92510",
        "expected": 18,
    },
    {
        "name": "Example 4",
        "s": "59414",
        "expected": 2018,
    },
]


def when_does_appear(s: int, is_puzzle: bool = False, do_print: bool = False) -> int:
    recipes = [3, 7]
    current_recipes = [0, 1]

    n = len(s) + (1 if is_puzzle else 0)

    while s not in "".join(str(i) for i in recipes[-n:]):
        recipes.extend(
            v
            for i, v in enumerate(divmod(sum(recipes[i] for i in current_recipes), 10))
            if i != 0 or v != 0
        )
        current_recipes = [(i + 1 + recipes[i]) % len(recipes) for i in current_recipes]

    return len(recipes) - n


run_tests_params(when_does_appear, tests_II)


Test Example 1 passed, for when_does_appear.
Test Example 2 passed, for when_does_appear.
Test Example 3 passed, for when_does_appear.
Test Example 4 passed, for when_does_appear.
Success


In [58]:
print(f"Part II: {when_does_appear("846021", is_puzzle=True)}")

Part II: 20368140


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>20368140</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
